<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left;padding10:3px;padding-top:10px;"></td>
<h1 style="white-space: nowrap"> Project Milestone 3</h1>
<h3 style="white-space: nowrap;margin:22px">  Applied Data Analysis (CS-401) </h3>
<hr style="clear:both">
<p style="font-size:1em; margin:2px; text-align:justify">
    <b>Description: </b>This Colab notebook is aimed at investigating the QuoteBank database provided in the course of Applied Data Analysis by Professor Robert West. The data consists of quotes from newspapers dated from 2015 to 2020 included.
</p>
<p style="font-size:0.95em; margin:0px"><b>Authors: </b> 
    <a href="mailto:lucas.brunschwig@epfl.ch"> Lucas Brunschwig</a>, 
    <a href="mailto:ioannis.mavrothalassitis@epfl.ch"> Ioannis Mavrothalassitis</a>, 
    <a href="mailto:axelle.piguet@epfl.ch"> Axelle Piguet</a>, 
    <a href="mailto:ester.simkova@epfl.ch"> Ester Simkova</a>

</p>
<p style="font-size:0.95em; margin:0px"><b>Data: </b> </p>
<ul> 
   <li> Raw data: <a href="https://zenodo.org/record/4277311#.YY0c_2DMJPY">here</a>. </li>
   <li> Processed Data: <a href="https://zenodo.org/record/4277311#.YY0c_2DMJPY">here</a> </li>
</ul>

<p  style="font-size:0.95em; margin:0px">
    <b>Last modification</b>: 16.12.2021 
</p>

<hr/>


<b><u>Table of contents</u></b>

0. [Loading](#0)
    
    0.1. [Installation](#0-1)
    
    0.2. [Libraries](#0-2)
    
    0.3. [Preliminary Data Exploration](#0-3)



1. [Bertopic Modeling](#1)  
    
    1.1. [Parameters](#1-1)
    
    1.2. [Data Loading](#1-2)
    
    1.3. [Bertopic Analysis](#1-3)


2. [Topics Analysis](#2)

    2.1 [Topics Selection](#1-1)
    
    2.2 [Visualisation over time](#1-2) 
    
    2.3 Removal of common english words




<hr/>

## 0. Loading <a class="anchor" id="0"></a>

Here we load the necessary libraries to run the notebook. One of the libraries necessit a specific version of some common libraries and you will have to restart the environment once installed.

### 0.1 Installation <a class="anchor" id="0-1"></a>

These two libraries are necessary to install the most exotic libraries to extract the topics of each quotes.

In [1]:
!pip install  sentence_transformers
!pip install bertopic

  Using cached sentence-transformers-2.1.0.tar.gz (78 kB)
  Using cached huggingface_hub-0.2.1-py3-none-any.whl (61 kB)
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=8a3232e3a5cb49bb5f21f9fde9d497f3ef7de9b0648f5e04e158e25df07cef16
  Stored in directory: c:\users\lucas\appdata\local\pip\cache\wheels\52\19\88\6625593382e23a926740e6fcee0f2df0a0de25766094842a28
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
  Using cached umap-learn-0.5.2.tar.gz (86 kB)
  Using cached hdbscan-0.8.27.tar.gz (6.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Prepar

### 0.2 Libraries <a class="anchor" id="0-2"></a>

In [2]:
##################### IMPORTS #####################

# NumPy to use arrays
import numpy as np

# DataFrame handling
import pandas as pd

# Data Decompression
import bz2

# Data Structure
import json

# Useful for chronological ordering
from datetime import datetime, date ,time 
import string


import os
from os import walk


### 0.3 Preliminary Data Exploration 

Just some preliminary exploration of the quotes that we have in the dataframe such that we get the global idea

<b>Format of a QUOTE</b>

Here, we will have a look at the shape of the data. The format of the original data-base is as follows:

* quoteID: a unique index that allow to select a quote
* quotation: the quote from beginning to end
* speakers: the most probable speakers (None, if unsure)
* qids: not sure
* date: date of parution to the miliseconds
* numOccurrences: the number of articles where the quote was mentionned
* probas: probability that different speakers said the quote
* urls: urls of articles that mentionned this quote
* phase: when was the data retrieved (not of interest to us)

<hr/>

In [8]:
# Load Data 
from data_loader import data_loader

files_to_load = "quotes-2015.json.bz2"
limits=10
chunk_ = 10
df_quotes = data_loader("data/"+files_to_load, limit = limits-1, chunksize_ = chunk_)



Loading file: "data/quotes-2015.json.bz2"
Beginning: Loading Quotes...
    Loading... 0.00 %
    Loading... 11.11 %
    Loading... 22.22 %
    Loading... 33.33 %
    Loading... 44.44 %
    Loading... 55.56 %
    Loading... 66.67 %
    Loading... 77.78 %
    Loading... 88.89 %
    Loading... 100.00 %
Number of quotes loaded in data/quotes-2015.json.bz2: 100


In [9]:
print("Data Shape (rows, columns):", np.shape(df_quotes))
print("Column names as depicted above:", df_quotes.columns)
df_quotes.head(4)

Data Shape (rows, columns): (100, 9)
Column names as depicted above: Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')


,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2015-08-31-000271,... a great day for veterans here in Littleton...,Jeanne Shaheen,[Q270316],2015-08-31 02:10:00,2,"[[Jeanne Shaheen, 0.742], [None, 0.2359], [Kel...",[http://www.unionleader.com/article/20150831/N...,E
1,2015-12-08-029916,How FFA scored 32 own goals in 18 months and C...,None,[],2015-12-08 00:00:00,2,"[[None, 0.563], [David Gallop, 0.437]]",[http://feeds.theroar.com.au/~r/theroar/~3/tZ3...,E
2,2015-09-10-000206,[ Amy ] was placed under an unacceptable amoun...,None,[],2015-09-10 21:18:59,1,"[[None, 0.9634], [Amy Robinson, 0.0366]]",[http://www.thefashionspot.com/buzz-news/lates...,E
3,2015-07-23-032707,How High Will These Numbers Go?,None,[],2015-07-23 14:57:49,3,"[[None, 0.9019], [Chubby Checker, 0.0981]]","[http://www.billboard.com/node/6641719, http:/...",E


In [10]:
print("One typical event:")
print()
df_quotes.loc[0] # example of one row

One typical event:



quoteID                                           2015-08-31-000271
quotation         ... a great day for veterans here in Littleton...
speaker                                              Jeanne Shaheen
qids                                                      [Q270316]
date                                            2015-08-31 02:10:00
numOccurrences                                                    2
probas            [[Jeanne Shaheen, 0.742], [None, 0.2359], [Kel...
urls              [http://www.unionleader.com/article/20150831/N...
phase                                                             E
Name: 0, dtype: object

**Search for a specific event and see if the quotes relate to it**

In [11]:
df_quotes.sort_values(by=['date'], inplace=True, ascending=True)

##### IMPORTANT: we take only a subset of quotes for a given time window
df_quotes = df_quotes[df_quotes.date.dt.month == 11] # The 2016 United States elections were held on Tuesday, November 8, 2016, let's look around this period
df_quotes = df_quotes[df_quotes.date.dt.day <= 12]
df_quotes = df_quotes[df_quotes.date.dt.day >= 5]
df_quotes.head()
print(np.shape(df_quotes))

(1, 9)


## 1. Bertopic Analysis

Here we load two python files which delocates the code and eases the reading of the notebook. 
* data_loader.py contains all the necessary functions such that we can load data from different functions. In order to process the data without overloading the ram and gpu to our disposition, we decided to limit the quotes of each month at 100'000 quotes. This choice was justified due to the facts that we found relevant topics in the results.
* bert.py contains all the necessary functions to perform the analysis by year and then by month. Each month is performed individually so that we can extract hot topics.

Limitations and Considerations:
* We don't use the whole dataset but this is not necessary as we only need to extract the hot topics. The sample that we extract is sufficient to significantly represent the topics that are of interest to us.
* The month sliding window is significantly large but given that we are interested in major topics we can estimate that it will be discussed over at least one month

**Initialization:**

In [12]:
# Initialize the Directory

path = os.getcwd()
print("Current Directory:",path)
print()

# Path needed to be ran
paths = ["results/bertopic/","data/"]

for path in paths:
    
    # Check if necessary paths exists
    if os.path.isdir(path): 
        print("Path %s already exists" % path)
        
    else:
        try:
            os.mkdir(path)
        except OSError:
            print ("Creation of the directory \"%s\" failed" % path)

        else:
            print ("Successfully created the directory %s " % path)
            if path=="data":
                print("You will need to dowload the data of the Quotebank dataset to load them")

Current Directory: C:\Users\Lucas\OneDrive\Bureau\ada-2021-project-adaventurers

Path results/bertopic/ already exists
Path data/ already exists


In [14]:
from data_loader import data_loader, clean_data
from bert import get_year_topics

# Parameters

year = 2020 # which year do you want to analyze
limits = 10 # the number of chunk to extract
chunk_ = 10 # the number of quotes for each chunk --> 10*100'000 = 1M quotes in total
threshold = 100000 # maximum number of quotes for one month

**Load Data:**

In [15]:
files_to_load = f"quotes-{year}.json.bz2"
data_year = pd.DataFrame()

print("\nBeginning: Loading File\n")

data_year = data_loader("data/"+files_to_load, limit = limits-1, chunksize_ = chunk_,thrs = threshold)

print("Done: Loading File\n")


Beginning: Loading File


Loading file: "data/quotes-2020.json.bz2"
Beginning: Loading Quotes...
    Loading... 0.00 %
    Loading... 11.11 %
    Loading... 22.22 %
    Loading... 33.33 %
    Loading... 44.44 %
    Loading... 55.56 %
    Loading... 66.67 %
    Loading... 77.78 %
    Loading... 88.89 %
    Loading... 100.00 %
Number of quotes loaded in data/quotes-2020.json.bz2: 100
Done: Loading File



**Bert Topic Analysis:**

In [16]:

print("\nBeginning: Bert Topic Analysis\n")

topic_assignation, prob_assignation, topic_list= get_year_topics(data_year)

topics_assignation = [element  for month in topic_assignation  for element in month ]
prob_assignation = [element  for month in prob_assignation  for element in month ]

data_year.sort_values(by=['date'], inplace=True, ascending=True)

data_year["topic_number"]=topics_assignation
data_year["topic_prob"]=prob_assignation


print("\nExample of the format")
print(data_year.columns)
print(data_year.head())
print()
print(topic_list.columns)
print(topic_list.head())

print("\nDone: Bert Topic Analysis\n")

#--------------------------------------------------------#
# Saving Topics

print("Saving Data:")
#data_year.to_pickle(f"results/bertopic/{year}_quotes_with_topics_{len(data_year)}.pkl")
#topic_list.to_pickle(f"results/bertopic/{year}topics_by_month_{len(data_year)}.pkl")
print("Done savings Data")

#--------------------------------------------------------#
# End of topic extractions
print("\nYou can find the results in \"results/bertopic/\" folder")


Beginning: Bert Topic Analysis

Year analysis:
    month: 1, number of quotes: 33
    month: 2, number of quotes: 30
    month: 3, number of quotes: 24
    month: 4, number of quotes: 13
    month: 5, number of quotes: 0
    month: 6, number of quotes: 0
    month: 7, number of quotes: 0
    month: 8, number of quotes: 0
    month: 9, number of quotes: 0
    month: 10, number of quotes: 0
    month: 11, number of quotes: 0
    month: 12, number of quotes: 0

Example of the format
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'topic_number', 'topic_prob'],
      dtype='object')
              quoteID                                          quotation  \
37  2020-01-03-001237  A lot of Americans [ were ] upset that this wa...   
45  2020-01-03-001453    a methodical combination of planning and chance   
23  2020-01-05-000384  A 25-year-old Sikh man, was killed by unidenti...   
51  2020-01-07-001821                        a 

## 2. Topics Analysis

Now that we extracted topics of each month and associate each quotes to a topic we will discuss here the results that we got. The goal is to extract the main topics that are specific to one event and represent it over time.

**List of the Bertopic Results that exists**

You might have a previous pandas version here it will upload it for you

In [1]:
import importlib
!pip install pandas==1.3.1
import pandas as pd
importlib.reload(pd)
print(pd.__version__)

1.3.1


### 2.1 Topics Exploration

In [14]:
mypath="results/bertopic/"
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break
for i in range(0,len(f),2):
    print(f"Index: {i},{i+1} |", f[i],f[i+1])

Index: 0,1 | 2015_quotes_with_topics_1000000.pkl 2015_topics_by_month_1000000.pkl
Index: 2,3 | 2016topics_by_month_737263.pkl 2016_quotes_with_topics_737263.pkl
Index: 4,5 | 2020topics_by_month_400000.pkl 2020_quotes_with_topics_400000.pkl


In [15]:
# Select the file of interest
i = 0
quote_file = f[i]
topic_file = f[i+1]

# Load results files
df_quotes = pd.read_pickle("results/bertopic/"+quote_file)
df_topics = pd.read_pickle("results/bertopic/"+topic_file)

In [61]:
#--------------------------------------------------------#
# Topics overview by months

# Format of the topic dataFrame
print(df_topics.columns)
print()
year = 2015

open(f"results/top_topics/{year}_top_topics.txt", 'w').close()
f = open(f"results/top_topics/{year}_top_topics.txt", "a")

# Selection of a specific month
for month in range(1,13):
    hot_topics = df_topics[df_topics.Month==month].sort_values(by="Count",ascending=False)
    f.write("#------------------------------------# \n")
    f.write(f"Month {month} \n\n")
    for topic_selected in hot_topics[1:10].Topic.values:

        # Quotes associate to the top topic
        # Will be change to target most probable quotes
        #print(df_quotes.columns)
        quotes_ = df_quotes[(df_quotes.date.dt.month ==month) & (df_quotes.topic_number==topic_selected)].sort_values(by="topic_prob",ascending=False).quotation.values

        f.write(f"Words associated to topic {topic_selected}: \n")
        f.write(str(hot_topics[hot_topics.Topic==topic_selected].Words.values))
        f.write("\n\n")
        f.write("Top quotes associated to the same topic: \n")
        
        for i,quote in enumerate(quotes_[0:5]):
            f.write(f"quote {i}: ")
            f.write(quote)
            f.write("\n")
        f.write("\n\n")
    
f.close()

Index(['Topic', 'Count', 'Name', 'Words', 'Month'], dtype='object')



### 2.2 Common Topic Analysis

Based on a systematic analysis of the top 20 topics. We selected the words that are the most common and discard them so that we can achieve more interesting topics as the top topics. We also use it to visualize the common topics and their evolution over time

### 2.3 Topics of Interest Extraction